In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup

In [2]:
# Launching the df
col_names = ["scrape_date", "calendar", "sowing", "spacing", "harvest", "companion", "avoid", "details"]
df = pd.read_csv("data.csv", names = col_names)
df.head(3)

,scrape_date,calendar,sowing,spacing,harvest,companion,avoid,details
0,2022-10-24 11:15:40.203136,"<div class=""row"" id=""calendar"">\n<div class=""c...",Sow in garden. \t\t\t\tSow seed at a depth app...,Space plants: 20 inches apart,Harvest in 7-8 weeks.,"Compatible with (can grow beside): Onions, cor...",NaN,Amaranth sp are frequently grown as flower pla...
1,2022-10-24 11:15:41.823406,"<div class=""row"" id=""calendar"">\n<div class=""c...",Easy to grow. Sow in garden. \t\t\t\tSow seed...,Space plants: 18 inches apart,Harvest in approximately 18 months. Angelica a...,Compatible with (can grow beside): Any herbs t...,NaN,Angelica is a biennial herb-growing the first ...
2,2022-10-24 11:15:43.608548,"<div class=""row"" id=""calendar"">\n<div class=""c...",Easy to grow. Sow in garden. \t\t\t\tSow seed...,Space plants: 63 - 79 inches apart,Harvest in 42-57 weeks.,Compatible with (can grow beside): Needs a lot...,NaN,Superthistles growing to 1.2-1.3m high with a ...


In [3]:
deg_pattern = "([\d]+)°F.+ ([\d]+)°F."
new_df = df.assign(
    min_soil_temp = df["sowing"].apply(lambda x: int(re.search(deg_pattern, x).group(1))),
    max_soil_temp = df["sowing"].apply(lambda x: int(re.search(deg_pattern, x).group(2)))
)

In [4]:
def find_val(input_val):
    pat1 = "(\d+)\s*-\s*(\d+)"
    pat2 = "(\d+)"
    search1 = re.search(pat1, input_val)
    if search1:
        min_val = int(search1.group(1))
        max_val = int(search1.group(2))
    else:
        search2 = re.search(pat2, input_val)
        if search2:
            search2 = search2.groups(1)[0]
            min_val = max_val = int(search2)
        else:
            min_val = max_val = -1
    return (min_val, max_val)

In [5]:
new_df = new_df.assign(
    min_space = df["spacing"].apply(find_val),
    max_space = df["spacing"].apply(find_val)
)
new_df["min_space"] = new_df["min_space"].apply(lambda x: x[0])
new_df["max_space"] = new_df["max_space"].apply(lambda x: x[1])

In [6]:
new_df = new_df.assign(
    min_harvest = df["harvest"].apply(find_val),
    max_harvest = df["harvest"].apply(find_val)
)
new_df["min_harvest"] = new_df["min_harvest"].apply(lambda x: x[0])
new_df["max_harvest"] = new_df["max_harvest"].apply(lambda x: x[1])

In [7]:
unit_pat = "\d+\s*([a-zA-z]+)\s*"
new_df["harvest_unit"] = new_df["harvest"].apply(lambda x: re.search(unit_pat, x).groups(1)[0])

In [8]:
def filter_companion(input_text):
    # Filter our None values and irrelevant texts
    if type(input_text) is np.float:
        return []
    if "bed" in input_text:
        return []
    start_index = input_text.find(":")
    if start_index == -1:
        return []
    start_index = input_text.find(":") + 1
    return [x.strip() for x in input_text[start_index:].strip().lower().split(",")]

In [9]:
new_df["companions"] = new_df["companion"].apply(filter_companion)

/var/folders/ml/3qpj1_6x4fj_gcl3ztrny2700000gn/T/ipykernel_25270/2404774270.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(input_text) is np.float:


In [10]:
new_df["avoid"] = new_df["avoid"].apply(filter_companion)

/var/folders/ml/3qpj1_6x4fj_gcl3ztrny2700000gn/T/ipykernel_25270/2404774270.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(input_text) is np.float:


In [19]:
decoder = {
    "alliums (chives": ["allium", "chives"],
    "and root vegetables": ["root vegetables"],
    "onions)": ["onions"],
    "onions) sunflower": ["onions", "sunflower"],
    "climbing (pole) beans": ["climbing beans", "pole beans"],
    "peppers (chilli": ["peppers", "chili"],
    "capsicum)": ["capsicum"],
    "eggplant (aubergine)": ["eggplant", "aubergine"],
    "alliums (garlic and onions)": ["alliums", "garlic", "onions"],
    "best grown away from vegetables": ["vegetables"],
    "cucurbit (cucumbers": ["cucurbit", "cucumbers"],
    "gourds)": ["gourds"],
}

In [20]:
new_avoids = []
for avoid in new_df["avoid"]:
    temp_set = set()
    # print(avoid)
    # lst = eval(avoid)
    for item in avoid:
        if item in decoder:
            temp_set.update(decoder[item])
        else:
            temp_set.add(item)
    new_avoids.append(temp_set)

In [ ]:
with open("avoids.txt", "w") as f:
    

In [26]:
new_avoids

[set(),
 set(),
 set(),
 {'garlic', 'onions', 'root vegetables'},
 set(),
 set(),
 {'allium', 'chives', 'florence fennel', 'garlic', 'leek', 'onions'},
 {'allium', 'chives', 'garlic', 'leek', 'onions', 'sunflower'},
 {'asparagus', 'carrots', 'spinach', 'sweetcorn'},
 set(),
 set(),
 {'aubergine',
  'capsicum',
  'chili',
  'climbing beans',
  'eggplant',
  'mustard',
  'peppers',
  'pole beans',
  'strawberry',
  'tomato'},
 {'aubergine',
  'capsicum',
  'climbing beans',
  'eggplant',
  'mustard',
  'peppers (chili',
  'pole beans',
  'strawberry',
  'tomato'},
 set(),
 {'aubergine',
  'capsicum',
  'climbing beans',
  'eggplant',
  'mustard',
  'parsnip',
  'peppers (chili',
  'pole beans',
  'strawberry',
  'tomato'},
 set(),
 set(),
 set(),
 {'beetroot', 'brassicas', 'dill', 'fennel', 'parsnips'},
 {'aubergine',
  'capsicum',
  'climbing beans',
  'eggplant',
  'mustard',
  'peppers (chili',
  'pole beans',
  'strawberry',
  'tomato'},
 set(),
 {'sweetcorn'},
 set(),
 set(),
 {'aub

In [25]:
df2 = pd.DataFrame(x=new_avoids)

TypeError: __init__() got an unexpected keyword argument 'x'

In [24]:
df2


,0,1,2,3,4,5,6,7,8,9
0,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None
3,garlic,onions,root vegetables,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
90,tomatoes,potatoes,None,None,None,None,None,None,None,None
91,potatoes,None,None,None,None,None,None,None,None,None
92,None,None,None,None,None,None,None,None,None,None
93,None,None,None,None,None,None,None,None,None,None


In [13]:
for val in new_df["avoid"]:
    print(val)

[]
[]
[]
['garlic', 'onions', 'and root vegetables']
[]
[]
['alliums (chives', 'leek', 'garlic', 'onions)', 'florence fennel']
['alliums (chives', 'leek', 'garlic', 'onions) sunflower']
['asparagus', 'carrots', 'sweetcorn', 'spinach']
[]
[]
['climbing (pole) beans', 'tomato', 'peppers (chilli', 'capsicum)', 'eggplant (aubergine)', 'strawberry', 'mustard']
['climbing (pole) beans', 'tomato', 'peppers (chili', 'capsicum)', 'eggplant (aubergine)', 'strawberry', 'mustard']
[]
['climbing (pole) beans', 'tomato', 'peppers (chili', 'capsicum)', 'eggplant (aubergine)', 'strawberry', 'mustard', 'parsnip']
[]
[]
[]
['parsnips', 'beetroot', 'dill', 'brassicas', 'fennel']
['climbing (pole) beans', 'tomato', 'peppers (chili', 'capsicum)', 'eggplant (aubergine)', 'strawberry', 'mustard']
[]
['sweetcorn']
[]
[]
['climbing (pole) beans', 'tomato', 'peppers (chilli', 'capsicum)', 'eggplant (aubergine)', 'strawberry', 'mustard']
[]
[]
['climbing (pole) beans', 'tomato', 'peppers (chili', 'capsicum)', 'e

In [67]:
def clean_calendar(dom):
    soup = BeautifulSoup(dom)
    table = soup.find_all("tr")
    row_num = 0
    s_res, t_res, p_res = [], [], []
    for row in table:
        if row_num == 0:
            pass
        row_vals = row.find_all("td")
        for i, val in enumerate(row_vals):
            val = val.text.strip()
            if val == "S":
                s_res.append(i+1)
            if val == "T":
                t_res.append(i+1)
            if val == "P":
                p_res.append(i+1)
        row_num += 1
    best_months["S"].append(s_res)
    best_months["T"].append(t_res)
    best_months["P"].append(p_res)
    return 

In [68]:
best_months = {"S": [], "T": [], "P": []}
new_df["calendar"].apply(clean_calendar)
new_df["S"] = best_months["S"]
new_df["T"] = best_months["T"]
new_df["P"] = best_months["P"]

0     None
1     None
2     None
3     None
4     None
      ... 
90    None
91    None
92    None
93    None
94    None
Name: calendar, Length: 95, dtype: object

In [73]:
new_df = new_df.drop(columns = ['calendar', 'sowing', 'spacing', 'harvest', 'companion','avoid'])

In [74]:
new_df.isnull().any()

scrape_date      False
details          False
min_soil_temp    False
max_soil_temp    False
min_space        False
max_space        False
min_harvest      False
max_harvest      False
harvest_unit     False
companions       False
S                False
T                False
P                False
dtype: bool

In [76]:
new_df.to_csv("partially_cleaned.csv")